In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
model = load_model('model.h5')

##load encoder and scaler

with open('lable_encoder_gender.pkl','rb') as file:
    lable_encoder_gender = pickle.load(file)
with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

2025-09-04 16:00:13.110474: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-04 16:00:13.110591: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-04 16:00:13.110608: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-04 16:00:13.110850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-04 16:00:13.110867: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
geo_encoder = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder , columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

/opt/anaconda3/envs/env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df['Gender'] = lable_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
input_df = pd.concat([input_df.drop('Geography', axis=1) , geo_encoder_df],axis=1 )

In [8]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_scaler = scaler.transform(input_df)

In [10]:
##prediction churn
predection = model.predict(input_scaler)

2025-09-04 16:00:22.757308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 761ms/step


In [11]:
predection_proba = predection[0][0]
predection_proba

0.06853172

In [12]:
if predection_proba > 0.5:
    print('coustomer likely to churn')
else:
    print('coustomer likely to not churn')

coustomer likely to not churn
